In [25]:

import bs4
from bs4 import BeautifulSoup
import lxml

with open("data/city.xml") as fp:
    soup = BeautifulSoup(fp, "xml")

In [38]:
def find_tag_children(node):
    tags=list()
    for child in node.children:
        if type(child)==bs4.element.Tag:
            tags.append(child)
    return tags

In [43]:
children=find_tag_children(soup.contents[0])
print(len(children))
for i in children:
    print(type(i))

37
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>


In [84]:
from collections import namedtuple

ValueT=namedtuple('ValueT',['cname','old_code','sng_code','gb_code','gb_code_path','status'])

def dfs(node:bs4.element.Tag,paths:list,values:list):
    cname=node.get('cname')
    old_code=node.get('name')
    sng_code=node.get('deprecated_code')
    gb_code=node.get('gb_code')

    gb_code_path=';'.join(paths) if len(paths)!=0 else None
    status=0 if node.get('status')=='active' else 1

    value_t=ValueT(cname,old_code,sng_code,gb_code,gb_code_path,status)
    if gb_code is not None and cname is not None:
        values.append(value_t)
    else:
        print(node)

    paths.append(gb_code)
    children=find_tag_children(node)
    for child in children:
        dfs(child,paths,values)
    paths.pop()
root =soup.contents[0]
paths=[]
values=[]
provinces=find_tag_children(root)
for province in provinces:
    dfs(province,paths,values)



<targeting cname="巢湖市" deprecated_code="103" id="2000504" name="341400" sname="巢湖" status="deprecated">
</targeting>


In [106]:
from typing import List
def make_sql(values:List[ValueT]):
    sql_template="""
    insert into rulelab_area_hierarchical_data
    (cname,old_code,sng_code,gb_code,gb_code_path,status)
    values {}
    """
    sql_values=[]
    for value_t in values:
        cname=f"'{value_t.cname}'"
        old_code=f"{value_t.old_code}" if value_t.old_code is not None else 'null'
        sng_code=f"{value_t.sng_code}" if value_t.sng_code is not None else 'null'
        gb_code=f"{value_t.gb_code}"
        gb_code_path=f"'{value_t.gb_code_path}'" if value_t.gb_code_path is not None else 'null'
        status=f"'{value_t.status}'"

        sql_values.append("\n("+",".join([cname,old_code,sng_code,gb_code,gb_code_path,status])+")")
    return sql_template.strip().format(",".join(sql_values))



sql=make_sql(values)


In [107]:
with open('data/insert.sql','w') as writer:
    writer.writelines(sql)

In [81]:
values[0]

ValueT(cname='中国未知', old_code='980000', sng_code='362', gb_code='0156', gb_code_path=None, status=0)